In [2]:
# Imports

!pip install tweepy

import boto3
import json
import tweepy as tw
import os
import string

# Access Twitter

consumer_key= 'RM9A4H79z1I2IYWA8FsqniUFt'
consumer_secret= 'IjtxnSWFvCWdqpbnDsU76eIPwvDZuRMPPOX9zJBZvGFx7PP27K'
access_token= '1230590210251067392-PYlf9tNwUzRllGlBa0MnCZHm6K5Hqv'
access_token_secret= 'Tsquav2CkiDG62nV3BgTFHkQte15Hz00Rc3VXSbjodz9w'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

# Use Amazon Comprehend

comprehend = boto3.client('comprehend', region_name='us-east-1')


# Extract and run sentiment analysis on the Tweets
def get_tweets(tag_name):
    """
    Uses the Twitter API and tweepy to scrape the 50 most recent
    tweets using a hashtag determined by the user
    Parameter: the tag to be determined in the hashtag
    Return: The dictionary containing the scores for each of the
    four different sentiments (Positive, Negative, Neutral, and Mixed)
    """
    tag_name = '#' + tag_name
    
    tweet_data = api.search(q = tag_name, lang = 'en', result_type = 'popular', count = 25, tweet_mode = 'extended')
    
    tweets = []
    for i in range(len(tweet_data)):
        status = api.get_status(tweet_data[i].id, tweet_mode = 'extended')
        if hasattr(status, 'retweeted_status'):  # Check if Retweet
            tweets.append(tweet_data[i].retweeted_status.full_text)
        else:
            tweets.append(tweet_data[i].full_text)
    return tweets


def run_comprehend(tweets):
    """
    Takes in a list containing the text of popular tweets
    returns a dictionary with the 4 sentiment values as the keys
    the values are the lists of each sentiment for each tweet
    e.g.
    {'Positive':[0.1, 0.2, ..., 0.8], ..., 'Mixed':[0.1, 0.2, ..., 0.8]}
    """
    i = 0
    
    # THIS WILL BE THE IMPORTANT COMMAND TO USE
    sentiments = comprehend.detect_sentiment(Text=tweets[i], LanguageCode='en')
    
    return sentiments

tweets = get_tweets('microsoft')
print(tweets)
print(run_comprehend(tweets))

     |████████████████████████████████| 147 kB 5.3 MB/s eta 0:00:01
['#Google and Alphabet CEO #SundarPichai has announced ₹135 crore funding support for #India, stating that he was ‘devastated to see the worsening COVID crisis’ in the country, while #Microsoft CEO #SatyaNadella has also extended help https://t.co/wi4NAXOYuy', '#Microsoft using #MixedReality for remote audits of #Azure Data Centers: #HoloLens allows #datacenter techs to provide "remote eyes." [)-) #AR #Hololens #HoloLens2 https://t.co/CafmX3I9Zk via @dcfrontier', "#Microsoft is on #TIME100Companies: an inaugural list of the world's most influential businesses [)-) https://t.co/lhSqmqYmf7 #MicrosoftMesh #MixedReality #VirtualReality #Azure #HoloLens2 #HoloLens", 'SharePoint servers are being picked at with high-risk, legitimate-looking, branded phish messages and preyed on by a #ransomware gang using an old bug.\n#CyberSecurity #Phishing #Malware #Microsoft @Cofense \nhttps://t.co/uWNZqLvC50', 'What #stocks are trending

NoCredentialsError: Unable to locate credentials

In [ ]:
import numpy as np

def get_relevance(num_likes, num_rt):
    """Factors in the number of likes and retweets a post has to determine
        how relevant that Tweet is (how many people see it/are affected by it)
        Parameters:
        - num_likes: the number of likes a Tweet has
        - num_rt: the number of retweets a Tweet has
    """
    
    # definitely need to play around with how exactly we want these numbers to work
    # maybe try to think of some test cases that make the numbers better
    likes_coeff = 0.1 * np.sqrt(num_likes)
    rt_coeff = 0.3 * np.sqrt(num_rt)
    return (likes_coeff + rt_coeff) / 100

    
likes = 12000
rt = 5000
get_relevancy(likes, rt)

In [ ]:
# What the "article" struct will look like:
# {
#    'content': all of the actual body of the tweet (not used for calculating the total score)
#    'score': {'Positive: ...', ..., 'Mixed': ...}
#    - dictionary of each sentiment average value
#    'relevance': decimal number between 1 and 100
# }

# NOTE: Looking like the compute_article_score() function will not really be needed actually
# since Comprehend can compute the sentiment of the entire tweet itself

def compute_total_score(articles_list):
    """Takes in a list of several article structures (10 articles or 25 Tweets)
    - article structure consists of:
        1. a sentiment dictionary (computed from compute_article_score)
        2. a reliability score (unclear yet how exactly determined, may not be used for Twitter)
            - NOT included for now (assuming tweets)
        3. a relevance score (if Twitter, will replace reliability and will consider # of likes and retweets)
    - return: a float value between 0 and 100 that reflects the sentiment score of the overall company
    """
    total = 0
    for article in articles_list:
        total += article['score'] * article['relevance']
    final_score = total / len(articles_list)
    return final_score

compute_total_score(articles)